In [2]:
#tipicas
import pandas as pd
import numpy as np

#datagen
from keras.preprocessing.image import ImageDataGenerator

#modelos
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255)
val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#creamos los generadores con batch de 30 fotos con el dir correpondiente
train_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/final_split/train"
val_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/final_split/val"
test_dir = "/home/vvdiaz1/code/vvdiaz1/automatic_tagging/raw_data/final_split/test"

train_generator = train_datagen.flow_from_directory(
                                train_dir,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

val_generator = train_datagen.flow_from_directory(
                                val_dir,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

test_generator = train_datagen.flow_from_directory(
                                test_dir,
                                target_size =(480, 360), # target_size = input image size
                                batch_size = 5,
                                class_mode ='categorical')

Found 31000 images belonging to 4 classes.
Found 8855 images belonging to 4 classes.
Found 4433 images belonging to 4 classes.


In [5]:
def load_own_model_master():
    model = Sequential()
    model.add(layers.Input(shape=(480,360,3), name='input'))
    model.add(layers.Conv2D(16, kernel_size=10, activation='relu'))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=8, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Conv2D(32, kernel_size=6, activation="relu"))
    model.add(layers.MaxPooling2D(3))

    model.add(layers.Flatten())
    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))

    model.compile(loss = 'categorical_crossentropy', 
                    optimizer = 'adam',
                    metrics = 'accuracy')
    
    return model

In [7]:
model = load_own_model_master()
model.summary()

2023-01-17 21:13:06.468218: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-01-17 21:13:06.469212: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-17 21:13:06.469861: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-HHE71063): /proc/driver/nvidia/version does not exist
2023-01-17 21:13:06.474213: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 471, 351, 16)      4816      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 157, 117, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 150, 110, 32)      32800     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 50, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 45, 31, 32)        36896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 15, 10, 32)       0

In [11]:
history = model.fit(
             train_generator,
             steps_per_epoch = 5,
            #  epochs = 30,
             callbacks = EarlyStopping(patience = 5, restore_best_weights=True),
             validation_data = val_generator,
             validation_steps = 15)

2023-01-17 21:35:05.636060: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 52902720 exceeds 10% of free system memory.


InvalidArgumentError: Graph execution error:

Detected at node 'categorical_crossentropy/softmax_cross_entropy_with_logits' defined at (most recent call last):
    File "/home/vvdiaz1/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/vvdiaz1/.pyenv/versions/3.10.6/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/vvdiaz1/.pyenv/versions/lewagon/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/vvdiaz1/.pyenv/versions/lewagon/lib/python3.10/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/vvdiaz1/.pyenv/versions/lewagon/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/vvdiaz1/.pyenv/versions/lewagon/lib/python3.10/site-packages/tornado/platform/asynci

In [ ]:
model.evaluate_generator(val_generator,verbose = 0)

In [ ]:
model.evaluate_generator(test_generator,verbose = 0)

### Ahora con transfer learning